In [1]:
# examples/python/Advanced/interactive_visualization.py

import numpy as np
import copy
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [56]:
# Load in the point cloud
pcd = o3d.io.read_point_cloud("PointCloud/PointCloud_box_a1.pcd") # path to point cloud file
print(f"Point cloud has {np.asarray(pcd.points).shape[0]} points.")

# (optional) keep an untouched copy
pcd_orig = copy.deepcopy(pcd)

Point cloud has 307200 points.


In [34]:
# visualize, window will pop up
# SHift + number will change color map, press number 1-4 to switch
o3d.visualization.draw_geometries([pcd_orig],
                                  zoom=0.79,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

### Divide into depth bands

In [ ]:
import numpy as np

# Four distance bands in metres ─ tweak to suit your scene
# Tilted angle of camera needs to be considered as well
bins = np.array([1, 450, 650, 750, np.inf])

# Any RGB you like, 0-1 floats.                                                                               
palette   = np.array([[0.8, 0.1, 0.1],   # band 0  (nearest)  – red
                      [0.9, 0.6, 0.1],   # band 1             – orange
                      [0.2, 0.7, 0.2],   # band 2             – green
                      [0.1, 0.4, 0.8]])  # band 3  (farthest) – blue

In [54]:
pts = np.asarray(pcd.points) # 3-D points

# Choose the metric you need
# 1. 3-D Euclidean distance from origin (sphere):
#dist = np.linalg.norm(pts, axis=1)         

# 2. Raw height (Z-axis):
dist = pts[:, 2]

# Digitise → which bin does each point fall into (0..3)?
band = np.digitize(dist, bins) - 1         # subtract 1 → 0-based

In [55]:
pcd.colors = o3d.utility.Vector3dVector(palette[band])
o3d.visualization.draw_geometries([pcd])

### Roatating view

In [20]:

def rotate_view(vis):
    ctr = vis.get_view_control()
    ctr.rotate(10.0, 0.0)
    return False

o3d.visualization.draw_geometries_with_animation_callback([pcd], rotate_view)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Segmentation

In [31]:
# simple depth segmentation and bounding box measurement
import numpy as np

points = np.asarray(pcd.points)
min_z, max_z = points[:, 2].min(), points[:, 2].max()
num_levels = 5                      # for example, split into 5 depth bands
z_step = (max_z - min_z) / num_levels

for i in range(num_levels):
    z_low = min_z + i * z_step
    z_high = z_low + z_step
    mask = (points[:, 2] >= z_low) & (points[:, 2] < z_high)
    level = pcd.select_by_index(np.where(mask)[0])

    # Calculate the bounding box for this level
    bbox = level.get_axis_aligned_bounding_box()
    print(f"Level {i + 1} bbox size:", bbox.get_extent())

Level 1 bbox size: [209.18073 204.07616 495.     ]
Level 2 bbox size: [764.68616 702.90647 495.     ]
Level 3 bbox size: [1019.87802  170.43698  344.     ]
Level 4 bbox size: [3.29761 0.07861 0.     ]
Level 5 bbox size: [168.7209  122.66535 241.     ]


In [32]:
o3d.visualization.draw_geometries([pcd])